# 在 PyTorch 中构建模型


## `torch.nn.Module` 和 `torch.nn.Parameter`

在本视频中，我们将讨论 PyTorch 提供的一些用于构建深度学习网络的工具。

除了 `Parameter`，我们在本视频中讨论的类都是 `torch.nn.Module` 的子类。`torch.nn.Module` 是 PyTorch 的基类，用于封装特定于 PyTorch 模型及其组件的行为。

`torch.nn.Module` 的一个重要行为是注册参数。如果某个 `Module` 子类具有学习权重，这些权重将表示为 `torch.nn.Parameter` 的实例。`Parameter` 类是 `torch.Tensor` 的子类，具有特殊行为：当它们被分配为 `Module` 的属性时，它们会被添加到该模块参数的列表中。这些参数可以通过 `Module` 类的 `parameters()` 方法访问。

以下是一个简单的例子：一个包含两个线性层和一个激活函数的简单模型。我们将创建它的一个实例，并查看其参数：

In [3]:
import torch

# 定义一个简单的模型，包含两个线性层和一个激活函数
class TinyModel(torch.nn.Module):
    
    def __init__(self):
        super(TinyModel, self).__init__()
        
        # 定义第一层线性层，输入维度为100，输出维度为200
        self.linear1 = torch.nn.Linear(100, 200)
        # 定义激活函数 ReLU
        self.activation = torch.nn.ReLU()
        # 定义第二层线性层，输入维度为200，输出维度为10
        self.linear2 = torch.nn.Linear(200, 10)
        # 定义 Softmax 函数
        self.softmax = torch.nn.Softmax()
    
    def forward(self, x):
        # 前向传播过程
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

# 创建模型实例
tinymodel = TinyModel()

print('模型结构:')
print(tinymodel)

print('\n\n单个层的结构:')
print(tinymodel.linear2)

print('\n\n模型的参数:')
for param in tinymodel.parameters():
    print(param)

print('\n\n某一层的参数:')
for param in tinymodel.linear2.parameters():
    print(param)

模型结构:
TinyModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)


单个层的结构:
Linear(in_features=200, out_features=10, bias=True)


模型的参数:
Parameter containing:
tensor([[ 0.0870, -0.0099, -0.0537,  ...,  0.0356, -0.0248, -0.0518],
        [-0.0452,  0.0734, -0.0553,  ...,  0.0116, -0.0245, -0.0068],
        [ 0.0557,  0.0161,  0.0208,  ..., -0.0874, -0.0654,  0.0855],
        ...,
        [ 0.0628, -0.0829,  0.0972,  ...,  0.0867,  0.0678, -0.0483],
        [-0.0333,  0.0051,  0.0614,  ..., -0.0153, -0.0023, -0.0703],
        [-0.0530,  0.0100, -0.0996,  ...,  0.0440,  0.0300,  0.0599]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0975, -0.0867,  0.0694, -0.0378, -0.0326, -0.0500,  0.0622, -0.0550,
        -0.0812, -0.0986, -0.0705, -0.0366, -0.0818, -0.0253, -0.0427, -0.0584,
         0.0715, -0.0246, -0.0128, -0.0529, -0.0602,  0.0468

上面展示了 PyTorch 模型的基本结构：有一个 `__init__()` 方法用于定义模型的各层和组件，`forward()` 方法用于前向计算。可以直接打印模型或其子模块，查看其结构。

## 常见层类型

### 线性层（Linear Layer）

最基本的神经网络层是*线性层*（或称全连接层）。该层的每个输入都会影响每个输出，影响程度由权重矩阵决定。如果模型有 *m* 个输入和 *n* 个输出，权重就是一个 *m × n* 的矩阵。例如：

In [4]:
# 定义一个线性层，输入维度为3，输出维度为2
lin = torch.nn.Linear(3, 2)
# 随机生成一个输入张量，形状为(1, 3)
x = torch.rand(1, 3)
print('输入:')
print(x)

print('\n\n权重和偏置参数:')
for param in lin.parameters():
    print(param)

# 前向传播，计算输出
y = lin(x)
print('\n\n输出:')
print(y)

输入:
tensor([[0.9474, 0.5117, 0.2794]])


权重和偏置参数:
Parameter containing:
tensor([[-0.0988,  0.1609,  0.2168],
        [-0.2138,  0.3528,  0.0032]], requires_grad=True)
Parameter containing:
tensor([-0.2971, -0.0877], requires_grad=True)


输出:
tensor([[-0.2478, -0.1089]], grad_fn=<AddmmBackward0>)


如果你用线性层的权重对输入 `x` 做矩阵乘法并加上偏置，就会得到输出向量 `y`。

另一个重要特性：当我们用 `lin.weight` 查看层的权重时，它是一个 `Parameter`（`Tensor` 的子类），并且默认会跟踪梯度。这是 `Parameter` 与普通 `Tensor` 的不同之处。

线性层在深度学习模型中非常常见，尤其是在分类模型的最后几层，最后一层的输出通常等于类别数。

### 卷积层（Convolutional Layer）

*卷积层*用于处理具有空间相关性的数据，常用于计算机视觉任务，用于提取局部特征。它也可用于 NLP 等其他领域，例如分析词语上下文。

我们在前面的 LeNet5 网络中已经见过卷积层的用法：

In [5]:
import torch.functional as F

# LeNet 卷积神经网络结构
class LeNet(torch.nn.Module):

    def __init__(self):
        super(LeNet, self).__init__()
        # 输入通道为1（灰度图），输出通道为6，卷积核大小为5x5
        self.conv1 = torch.nn.Conv2d(1, 6, 5)
        self.conv2 = torch.nn.Conv2d(6, 16, 3)
        # 全连接层
        self.fc1 = torch.nn.Linear(16 * 6 * 6, 120)  # 6*6来自图像尺寸
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        # 2x2的最大池化
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 如果池化窗口是正方形，可以只写一个数字
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))  # 展平成一维向量
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # 除去batch维度的所有维度
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

让我们分解一下这个模型中卷积层的工作原理。从 `conv1` 开始：

* LeNet5 旨在处理 1x32x32 的黑白图像。**卷积层构造函数的第一个参数是输入通道数。** 在这里是 1。如果我们构建这个模型来处理 3 色通道，它将是 3。
* 卷积层就像一个窗口，扫描图像，寻找它识别的模式。这些模式称为*特征*，卷积层的一个参数是我们希望它学习的特征数量。**构造函数的第二个参数是输出特征数。** 在这里，我们要求我们的层学习 6 个特征。
* 刚才，我将卷积层比作一个窗口——但窗口有多大？**第三个参数是窗口或*核*大小。** 在这里，“5”表示我们选择了一个 5x5 的核。（如果您想要一个高度不同于宽度的核，可以为此参数指定一个元组——例如，`(3, 5)` 来获得一个 3x5 的卷积核。）

卷积层的输出是一个*激活图*——输入张量中特征存在的空间表示。`conv1` 将为我们提供一个 6x28x28 的输出张量；6 是特征数，28 是我们图的高度和宽度。（28 来自于在 32 像素行上扫描 5 像素窗口时，只有 28 个有效位置。）

然后我们将卷积的输出通过 ReLU 激活函数（稍后会详细介绍激活函数），然后通过最大池化层。最大池化层将激活图中彼此接近的特征分组在一起。它通过减少张量，将输出中的每个 2x2 单元组合并为一个单元，并为该单元分配输入的 4 个单元中的最大值。这样我们就得到了激活图的低分辨率版本，尺寸为 6x14x14。

我们的下一个卷积层 `conv2` 期望有 6 个输入通道（对应于第一层寻找的 6 个特征），有 16 个输出通道，以及一个 3x3 的核。它输出一个 16x12x12 的激活图，这个激活图再次通过最大池化层减少到 16x6x6。在将此输出传递给线性层之前，它被重塑为一个 16 * 6 * 6 = 576 元素的向量，以供下一层使用。

有用于处理 1D、2D 和 3D 张量的卷积层。卷积层构造函数还有许多其他可选参数，包括步幅长度（例如，仅扫描每隔一个或每隔三个位置）输入中的填充（因此您可以扫描到输入的边缘）等等。有关更多信息，请参阅[文档](https://pytorch.org/docs/stable/nn.html#convolution-layers)。

### 循环层

*循环神经网络*（或 *RNN*）用于处理序列数据——从科学仪器的时间序列测量到自然语言句子再到 DNA 核苷酸。RNN 通过维护一个*隐藏状态*来实现这一点，该隐藏状态充当它迄今为止在序列中看到的内容的记忆。

RNN 层的内部结构——或其变体 LSTM（长短时记忆）和 GRU（门控循环单元）——适度复杂，超出了本视频的范围，但我们将向您展示一个基于 LSTM 的词性标注器（一个分类器类型，告诉你一个词是名词、动词等）：

In [6]:
# 定义一个基于 LSTM 的词性标注器
class LSTMTagger(torch.nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        # 定义词嵌入层
        self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)

        # LSTM 层，输入为词嵌入，输出为隐藏状态
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)

        # 从隐藏状态空间映射到标签空间的线性层
        self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        # 词嵌入
        embeds = self.word_embeddings(sentence)
        # LSTM 前向传播
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        # 映射到标签空间
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        # 计算标签分数
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

构造函数有四个参数：

* `vocab_size` 是输入词汇表中的单词数量。每个单词都是一个 `vocab_size` 维空间中的一个独热向量（或单位向量）。
* `tagset_size` 是输出集合中的标签数量。
* `embedding_dim` 是词汇表的*嵌入*空间的大小。嵌入将词汇表映射到一个低维空间，其中具有相似含义的单词在空间中彼此接近。
* `hidden_dim` 是 LSTM 的记忆大小。

输入将是一个句子，其中的单词表示为独热向量的索引。嵌入层会将这些降维到一个 `embedding_dim` 维空间。LSTM 接受这个嵌入序列并对其进行迭代，生成一个长度为 `hidden_dim` 的输出向量。最终的线性层充当分类器；对最终层的输出应用 `log_softmax()` 将输出转换为归一化的估计概率集，表示给定单词映射到给定标签的概率。

如果您想看到这个网络的实际效果，请查看 pytorch.org 上的[序列模型和 LSTM 网络](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html)教程。

### Transformer

*Transformer* 是多用途网络，已通过 BERT 等模型在 NLP 中占据了最先进的地位。Transformer 架构的讨论超出了本视频的范围，但 PyTorch 有一个 `Transformer` 类，允许您定义 Transformer 模型的整体参数——注意力头的数量、编码器和解码器层的数量、dropout 和激活函数等。（您甚至可以通过这个单一类构建 BERT 模型，只需设置正确的参数！）`torch.nn.Transformer` 类还有类来封装单个组件（`TransformerEncoder`、`TransformerDecoder`）和子组件（`TransformerEncoderLayer`、`TransformerDecoderLayer`）。有关详细信息，请查看 Transformer 类的[文档](https://pytorch.org/docs/stable/nn.html#transformer)以及 pytorch.org 上的相关[教程](https://pytorch.org/tutorials/beginner/transformer_tutorial.html)。

## 其他层和函数

### 数据操作层

还有其他层类型在模型中执行重要功能，但它们本身不参与学习过程。

**最大池化**（及其双胞胎最小池化）通过组合单元来减少张量，并将输入单元的最大值分配给输出单元。（我们已经看到了这一点）例如：

In [7]:
# 随机生成一个张量，形状为(1, 6, 6)
my_tensor = torch.rand(1, 6, 6)
print(my_tensor)

# 定义一个最大池化层，池化核大小为3x3
maxpool_layer = torch.nn.MaxPool2d(3)
print(maxpool_layer(my_tensor))

tensor([[[0.7463, 0.0656, 0.5229, 0.9534, 0.5488, 0.8549],
         [0.1664, 0.3310, 0.7611, 0.5432, 0.3281, 0.0724],
         [0.7962, 0.8198, 0.0867, 0.3030, 0.6045, 0.9348],
         [0.4988, 0.7893, 0.9697, 0.7692, 0.6595, 0.0251],
         [0.4757, 0.6577, 0.9590, 0.4400, 0.1298, 0.5885],
         [0.4843, 0.4012, 0.8723, 0.8708, 0.0599, 0.9835]]])
tensor([[[0.8198, 0.9534],
         [0.9697, 0.9835]]])


如果仔细观察上面的值，您会发现最大池化输出中的每个值都是 6x6 输入的每个象限的最大值。

**归一化层**在将一个层的输出馈送到另一个层之前重新居中并归一化它。对中间张量进行居中和缩放具有许多有益的效果，例如让您在没有梯度爆炸/消失的情况下使用更高的学习率。

In [8]:
# 随机生成一个张量，并添加缩放因子和偏移
my_tensor = torch.rand(1, 4, 4) * 20 + 5
print(my_tensor)

# 计算张量的均值
print(my_tensor.mean())

# 定义一个归一化层
norm_layer = torch.nn.BatchNorm1d(4)
normed_tensor = norm_layer(my_tensor)
print(normed_tensor)

# 计算归一化后的张量的均值
print(normed_tensor.mean())


tensor([[[16.5149, 12.5266,  8.3586,  7.6942],
         [19.5170, 22.5475, 23.8300,  9.7521],
         [16.5405, 13.5382, 17.3652, 18.8554],
         [14.8051, 24.7900, 23.4329, 18.7057]]])
tensor(16.7984)
tensor([[[ 1.4773,  0.3532, -0.8216, -1.0089],
         [ 0.1098,  0.6592,  0.8918, -1.6608],
         [-0.0177, -1.5656,  0.4075,  1.1758],
         [-1.4223,  1.1009,  0.7580, -0.4366]]],
       grad_fn=<NativeBatchNormBackward0>)
tensor(-2.3842e-07, grad_fn=<MeanBackward0>)


运行上面的单元格，我们为输入张量添加了一个大的缩放因子和偏移；您应该会看到输入张量的 `mean()` 大约在 15 附近。将其通过归一化层后，您可以看到值变小，并且围绕零分布——实际上，均值应该非常小（> 1e-8）。

这很有益，因为许多激活函数（下面将讨论）在 0 附近具有最强的梯度，但有时会因输入驱动它们远离零而导致梯度消失或爆炸。将数据保持在最陡梯度区域附近通常意味着更快、更好的学习和更高的可行学习率。

**Dropout 层**是一种鼓励模型中*稀疏表示*的工具——即推动它使用更少的数据进行推理。

Dropout 层通过在训练期间随机设置输入张量的部分来工作——Dropout 层在推理时始终关闭。这迫使模型学习针对这个被屏蔽或减少的数据集。例如：

In [9]:
# 随机生成一个张量
my_tensor = torch.rand(1, 4, 4)

# 定义一个 Dropout 层，丢弃概率为0.4
dropout = torch.nn.Dropout(p=0.4)
print(dropout(my_tensor))
print(dropout(my_tensor))

tensor([[[0.1656, 0.8117, 0.0000, 0.0000],
         [0.0000, 1.4981, 0.1174, 0.0000],
         [1.0748, 0.0000, 0.8687, 0.2071],
         [0.2186, 1.3512, 0.1603, 1.1449]]])
tensor([[[0.0000, 0.8117, 0.0000, 1.4562],
         [0.0000, 1.4981, 0.0000, 0.0000],
         [1.0748, 0.0000, 0.0000, 0.0000],
         [0.2186, 1.3512, 0.0000, 1.1449]]])


上面，您可以看到 Dropout 对样本张量的影响。您可以使用可选的 `p` 参数设置单个权重丢弃的概率；如果不设置，它默认为 0.5。

### 激活函数

激活函数使深度学习成为可能。神经网络实际上是一个程序——有许多参数——*模拟数学函数*。如果我们所做的只是重复地通过层权重乘以张量，我们只能模拟*线性函数；*此外，拥有许多层将毫无意义，因为整个网络可以简化为单个矩阵乘法。在层之间插入*非线性*激活函数是使深度学习模型能够模拟任何函数而不仅仅是线性函数的原因。

`torch.nn.Module` 有封装所有主要激活函数的对象，包括 ReLU 及其许多变体、Tanh、Hardtanh、sigmoid 等。它还包括其他函数，例如 Softmax，这些函数在模型的输出阶段最有用。

### 损失函数

损失函数告诉我们模型的预测与正确答案的距离。PyTorch 包含多种损失函数，包括常见的 MSE（均方误差 = L2 范数）、交叉熵损失和负似然损失（对分类器有用）等。

## 高级：替换层